In [2]:
import invoiceTool
import uuid
import pandas as pd
pd.set_option('display.max_rows', 500)

peers = [invoiceTool.peer(addr=invoiceTool.addr1, crt=invoiceTool.crt1),
         invoiceTool.peer(addr=invoiceTool.addr2, crt=invoiceTool.crt2)]

# Load some invoices, add hash values

In [2]:
invoices = pd.read_csv('/home/jovyan/work/supermarketsales.csv')  
invoices.set_index('Invoice ID', inplace=True)
invoices['hash'] = invoices.apply(lambda x: hash(tuple(x)), axis = 1)

In [3]:
invoices.head()

Branch       City Customer type  Gender            Product line  \
Invoice ID                                                                    
750-67-8428      A     Yangon        Member  Female       Health and beauty   
226-31-3081      C  Naypyitaw        Normal  Female  Electronic accessories   
631-41-3108      A     Yangon        Normal    Male      Home and lifestyle   
123-19-1176      A     Yangon        Member    Male       Health and beauty   
373-73-7910      A     Yangon        Normal    Male       Sports and travel   

             Unit price  Quantity   Tax 5%     Total       Date   Time  \
Invoice ID                                                               
750-67-8428       74.69         7  26.1415  548.9715   1/5/2019  13:08   
226-31-3081       15.28         5   3.8200   80.2200   3/8/2019  10:29   
631-41-3108       46.33         7  16.2155  340.5255   3/3/2019  13:23   
123-19-1176       58.22         8  23.2880  489.0480  1/27/2019  20:33   
373-73-7910       86.31         7  30.2085  634.3785   2/8/2019  10:37   

                 Payment    cogs  gross margin percentage  gross income  \
Invoice ID                                                                
750-67-8428      Ewallet  522.83                 4.761905       26.1415   
226-31-3081         Cash   76.40                 4.761905        3.8200   
631-41-3108  Credit card  324.31                 4.761905       16.2155   
123-19-1176      Ewallet  465.76                 4.761905       23.2880   
373-73-7910      Ewallet  604.17                 4.761905       30.2085   

             Rating                 hash  
Invoice ID                                
750-67-8428     9.1  8252117794214139958  
226-31-3081     9.6  -333208600729903974  
631-41-3108     7.4 -4675384210355374297  
123-19-1176     8.4  2881726942978795534  
373-73-7910     5.3  6907212254428517984

In [5]:
def createInvoiceFromData(index=1,id = 41):
    row = invoices.iloc[index]
    invoiceid = row.name
    tax = row['Tax 5%']
    netto = row['Total'] - tax
    hashvalue = row['hash']
    return invoiceTool.createInvoice(id=id,invoiceNumber=invoiceid,vat=tax,netto=netto,countryOrigin='US',hash=hashvalue)

for x in range(0,99):
    createInvoiceFromData(index=x,id = 400 + x)

In [ ]:
#df = invoiceTool.getInvoicesAsDF()
#df.head()

In [19]:
import ipywidgets as widgets
#from IPython.display import display
from IPython.display import Javascript, display

import invoiceTool
import uuid


#def createInvoice(word1,word2):
#    print("test")
#    invoiceTool.createInvoice(id=word1,invoiceNumber=word2,sell="True")

class invoice_widgets():
    layout = widgets.Layout(width='auto', height='40px') #set width and height
    style = {'description_width': '150px'}
    layout = {'width': '400px'}

    def helper(a,b,c):
            print (a)

    def setvalues1(netto='', vat='') :
        print(netto)
        self.netto.value = netto
        self.vat.value = vat
            
    def createInvoice(self,word1,word2):
        #invoiceTool.createInvoice(id=word1,invoiceNumber=word2,sell="True")
        res=invoiceTool.createInvoice(id=word1,invoiceNumber=word2,received="false",sell="false")
        print(word2 + "=" + str(res))
        return "ok"
            
    #def __init__(self, param1 = str(uuid.uuid4()), param2 = ''):
    def __init__(self, param1 = '', param2 = '', peers = None):
        self.peers = peers
        #self.TextField1= widgets.Text(value='', description='Client List Name:',disabled=False, style=self.style, layout=self.layout)
        self.ledger = widgets.Dropdown(description = 'Ledger',options=['mychannel'], style=self.style, layout=self.layout)
        self.peer = widgets.Dropdown(description = 'Peer',options=[peers[0].addr,peers[1].addr], style=self.style, layout=self.layout)
        self.p1_text = widgets.Text(description = 'id',value = param1,disabled=False, style=self.style, layout=self.layout)
        # Rechnugsnnummer
        self.p2_text = widgets.Text(description = 'Rechnungsnummer',value = param2, style=self.style, layout=self.layout)
        # Country_Invoice Erstellung
        self.country_origin = widgets.Text(description = 'country (origin)',value = 'DE', style=self.style, layout=self.layout)
        self.country_receiver = widgets.Text(description = 'country (receiver)',value = '', style=self.style, layout=self.layout)
        
        # Invoice erhalten
        self.received = widgets.Checkbox(value=False, description='received (invoice)', style=self.style, layout=self.layout)

        self.order_received = widgets.Checkbox(value=False, description='received (order)', style=self.style, layout=self.layout)
        
        # Forderungsabtritt boolean (ID)
        self.forderungsabtritt = widgets.Checkbox(value=False, description='Forderungsabtritt', style=self.style, layout=self.layout)
       
        self.paid = widgets.Checkbox(value=False, description='Forderung bezahlt', style=self.style, layout=self.layout)
        # Geld erhalten auch von j anders / Forderung erhalten  (ID)
        self.forderungErhaltenVon = widgets.Text(description = 'Forderung erhalten von',value = '', style=self.style, layout=self.layout)
        self.steuerbefreiungsgrund = widgets.Dropdown(description = 'Steuerbefreieungsgrund',options=['','Ausfuhr'], style=self.style, layout=self.layout)
 
        #Umsatzsteuer abgeführt (Boolean) (wird von Finanzbehörde gesetzt)
        self.tax_received = widgets.Checkbox(value=False, description='Umsatzsteuer abgeführt', style=self.style, layout=self.layout)    
    
        # VAT- amount
        self.vat = widgets.Text(description = 'VAT',value = '', style=self.style, layout=self.layout)
        self.netto = widgets.Text(description = 'Netto',value = '', style=self.style, layout=self.layout)
        self.hash_text = widgets.Text(description = 'Hash (invoice)',value = '', style=self.style, disabled=True, layout=self.layout)
        
        self.invoicedata_actions = {self.p2_text, self.vat, self.netto, self.country_origin,self.country_receiver}
        for action in self.invoicedata_actions:
            action.disabled = True
        
        self.seller_actions = {self.forderungErhaltenVon, self.forderungsabtritt}
        
        self.notuser_actions = {self.hash_text,self.steuerbefreiungsgrund,
                                self.tax_received, 
                                self.forderungsabtritt,
                               self.forderungErhaltenVon }
        for action in self.notuser_actions:
            action.disabled = True
        
        self.blocked = False
        self.user_actions = {self.received,self.order_received, self.paid}
        for action in self.user_actions:
            action.disabled = True
        
        #self.received.on_submit(self.handle_submit) 
        self.received.observe(self.changed)
        self.order_received.observe(self.changed)
        self.peer.observe(self.changedpeer)
        
        self.p1_text.on_submit(self.handle_submit)
        self.p2_text.on_submit(self.handle_submit)
        display(self.ledger, self.peer,
                self.hash_text, self.p1_text, self.p2_text, self.vat, self.netto, self.country_origin,self.country_receiver, self.received,self.order_received,  
                self.forderungsabtritt, self.paid, self.forderungErhaltenVon,
                self.steuerbefreiungsgrund,self.tax_received)
        
    def changedpeer(self, change):   
        if change['type'] == 'change' and change['name'] == 'value':
            #print('peer')
            if self.peer.value == self.peers[0].addr:
                for action in self.seller_actions:
                    action.disabled = True
                for action in self.user_actions:
                    #print(str(action))
                    if action.value == False:
                        action.disabled = False
                    else:
                        action.disabled = True
                        #print(action)
            else:
                for action in self.user_actions:
                    action.disabled = True
                self.forderungErhaltenVon.disabled = False    
                for action in self.seller_actions:
                    #print(str(action))
                    if action.value == False:
                        action.disabled = False
                    else:
                        action.disabled = True   
        
    def changed(self, change):    
        #print(b['description'])
        #print(b.description)
        if self.blocked == True:
            print('blocked')
            return
        
        if change['type'] == 'change' and change['name'] == 'value':
            print ("changed to %s" % change['new'])
            print(change['owner'].description)
            #if change['owner'] == self.received:
            if change['owner'] in self.user_actions:    
                print('received')
                change['owner'].disabled = True
                if change['owner'] == self.received:
                    invoiceTool.markAsReceived(self.p1_text.value)
                elif change['owner'] == self.order_received:
                    invoiceTool.markOrderReceived(self.p1_text.value)
                    
        
    def handle_submit(self, text):
        #print ("Submitting:")
        #print ("Text " + str(text.value))
        key = self.p1_text.value
        word2 = self.p2_text.value
        #print(self.p1_text.value + "=" + word2)
        
        val = invoiceTool.queryInvoice(id=key)
        print (val)
        if val is None:
            self.w = self.createInvoice(key,word2)
            self.p1_text.value = str(int(key) + 1)
            self.p2_text.value = ''
        else:
            #netto=str(val['netto'])
            #self.setvalues1(netto=netto)
            print("key=" + key)
            self.blocked = True
            
            self.netto.value = str(val['netto'])
            self.vat.value = str(val['vat'])
            self.p2_text.value = str(val['invoiceNumber'])
            self.country_origin.value = str(val['countryOrigin'])
            self.country_receiver.value = str(val['countryReceiver'])
            self.hash_text.value = str(val['hash'])
            self.forderungErhaltenVon.value = str(val['forderungErhaltenVon'])
            self.steuerbefreiungsgrund.value = str(val['steuerbefreiungsgrund'])
            
            self.received.value       = val['received']
            self.order_received.value = val['receivedOrder']
            
            #self.paid.value = str(val['forderungBezahlt'])
            #self.tax_received.value = str(val['umsatzsteuerAbgefuehrt'])
            #self.forderungsabtritt.value = str(val['sell'])
            self.blocked = False
            
            print(self.peer.value, self.peers[0].addr)
            
            if self.peer.value == self.peers[0].addr:
                for action in self.seller_actions:
                    action.disabled = True
                for action in self.user_actions:
                    #print(str(action))
                    if action.value == False:
                        action.disabled = False
                    else:
                        action.disabled = True
                        #print(action)
            else:
                for action in self.user_actions:
                    action.disabled = True
                self.forderungErhaltenVon.disabled = False    
                for action in self.seller_actions:
                    #print(str(action))
                    if action.value == False:
                        action.disabled = False
                    else:
                        action.disabled = True   
                    
            #print ('fill:' + str(val))
            #return None
            self.w='true'
       
        return self.w

In [18]:
f = invoice_widgets(peers=peers)

Dropdown(description='Ledger', layout=Layout(width='400px'), options=('mychannel',), style=DescriptionStyle(de…

Dropdown(description='Peer', layout=Layout(width='400px'), options=('peer0.org1.example.com:7051', 'peer0.org2…

Text(value='', description='Hash (invoice)', disabled=True, layout=Layout(width='400px'), style=DescriptionSty…

Text(value='', description='id', layout=Layout(width='400px'), style=DescriptionStyle(description_width='150px…

Text(value='', description='Rechnungsnummer', disabled=True, layout=Layout(width='400px'), style=DescriptionSt…

...

Checkbox(value=False, description='Forderungsabtritt', disabled=True, layout=Layout(width='400px'), style=Desc…

Checkbox(value=False, description='Forderung bezahlt', disabled=True, layout=Layout(width='400px'), style=Desc…

Text(value='', description='Forderung erhalten von', disabled=True, layout=Layout(width='400px'), style=Descri…

Dropdown(description='Steuerbefreieungsgrund', disabled=True, layout=Layout(width='400px'), options=('', 'Ausf…

Checkbox(value=False, description='Umsatzsteuer abgeführt', disabled=True, layout=Layout(width='400px'), style…

In [ ]:
df = invoiceTool.getInvoicesAsDF()
df[df['record.invoiceNumber'].str.contains("s")]